<a href="https://colab.research.google.com/github/sitopp/YUFO/blob/main/yuho_ChatGPT_with_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChainを使ってChatGPTに有価証券報告書を読み解いてもらう




[事前準備]
* 金融庁のサイトから有価証券報告書をダウンロードし、Google Driveにアップ

[本体]
1.   OpenAPIキーの設定と、関連ライブラリのインストールとインポート
2.   CSVをロードし、LangChainを使ってチャンクに分割する
3.   テキスト情報を数値化（Embedding)し、保存する
4.   データ取得関数のセットアップ
5.   チャットbotの作成


こちらのスクリプトは、@windows222(まさ まさ)さんの記事 https://qiita.com/windows222/items/232f05bafa95a9c8874e を参考にして独自の情報を追加しました。





# 0. 金融庁のサイトから有価証券報告書をダウンロードし、Google Driveにアップ



1.  金融庁EDINETにアクセスし、読み解きしたい有価証券報告書のcsvファイルをダウンロード
https://disclosure2.edinet-fsa.go.jp/week0010.aspx

2.   GoogleDriveの「マイドライブ」以下にフォルダを作成し、csvをアップロード
  * この構成を前提に進めます。
      * マイドライブ > yuho > before : csvをアップロード
      * マイドライブ > yuho > encoded
      * マイドライブ > yuho > after

3.  必ず以下の「file_id」で指定するファイル名を変更。
必要に応じてパス指定も変更する。

注意：このスクリプトでは、1つの有価証券報告書のみ読み込ませるようになってます。複数の企業に対応する場合は拡張が必要です。





In [ ]:
# ファイル名指定
# file_id = 'jpcrp040300-q3r-001_E33601-000_2023-06-30_01_2023-08-14'
file_id = 'jpcrp040300-q3r-001_E36070-000_2023-05-31_01_2023-07-14' #アララさん
# file_id = 'jpcrp040300-q3r-001_E37815-000_2023-03-31_01_2023-05-12' #unerry


# パス指定
before_file = '/content/drive/MyDrive/yuho/before/'+file_id+'.csv'
encoded_file = '/content/drive/MyDrive/yuho/encoded/'+file_id+'.csv'
after_file='/content/drive/MyDrive/yuho/after/'+file_id+".csv"


# 1. OpenAPIキーの設定と、関連ライブラリのインストール


Google ドライブをマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

ライブラリのインストール


In [ ]:
!pip install -q langchain==0.0.150 pypdf pandas matplotlib tiktoken textract transformers openai faiss-cpu requests beautifulsoup4

コード書く

In [ ]:
import os
import pandas as pd

import requests
import textract
# from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from transformers import GPT2TokenizerFast
# from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain


以下にOpenAIから取得したキーを設定します。キーは以下のURLから取得することができます。  
（アカウントの作成が必要です）  
https://platform.openai.com/account/api-keys



In [ ]:
# ここにOpenAIから取得したキーを設定します。
os.environ["OPENAI_API_KEY"] = 'ここに書く'

# 2. CSVをロードし、LangChainを使ってチャンクに分割する





金融庁の有価証券報告書csvファイルはutf-16leで書かれている。
このままだと使えないのでutf-8にエンコーディングする。





In [ ]:


# UTF-16LE エンコーディングでファイルを開く
with open(before_file, 'r', encoding='utf-16le') as file:
    content = file.read()

# UTF-8 エンコーディングで内容を新しいファイルに書き込む
with open(encoded_file, 'w', encoding='utf-8') as file:
    file.write(content)

# ファイルを読み込む
with open(encoded_file, 'r') as f:
    text = f.read()



トークンをカウントし、スプリットする

In [ ]:
# Create function to count tokens
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

def count_tokens(text: str) -> int:
    return len(tokenizer.encode(text))

# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 512,
    chunk_overlap  = 24,
    length_function = count_tokens,
)

chunks = text_splitter.create_documents([text])


In [ ]:
print (chunks)

# 3. テキスト情報を数値化（Embedding)し、保存する
ここでは、これまでに読み込んだ3つの情報をVector storeに保存します。。
Vector storeはいくつも種類がありますが、今回はFacebook製のFAISSを利用します。  


> 利用可能なVectorStoreはこちらを参照してください。
https://python.langchain.com/en/latest/modules/indexes/vectorstores.html#




In [ ]:
# Get embedding model
embeddings = OpenAIEmbeddings()

#  vector databaseの作成
db = FAISS.from_documents(chunks, embeddings)

# 4. データ取得関数のセットアップ

次に、先ほど作ったFAISSのvector storeにqueryを投げてどの様なデータが返ってくるかを確認します。

In [ ]:
query = "この企業の株式発行数を教えて"
# FAISSに対して検索。検索は文字一致ではなく意味一致で検索する(Vector, Embbeding)
docs = db.similarity_search(query)
docs # ここで関係のありそうなデータが返ってきていることを確認できる

## ユーザからのクエリを使って関連するデータを取得できる様にQAチェインを作成する

ここで、これまでにロードしたドキュメントに書かれた情報に関する質問を投げてみて、期待する結果が返ってくるかどうかを確認します。


>  以下コード内のtemperatureを変更することにより、情報の精度を上げることができます。(0-2までの値で指定）0にした場合、質問内容の回答がはっきりわからない場合はI don't knowと言われます。。



In [ ]:
# 得られた情報から回答を導き出すためのプロセスを以下の4つから選択する。いずれもProsとConsがあるため、適切なものを選択する必要がある。
# staffing ... 得られた候補をそのままインプットとする
# map_reduce ... 得られた候補のサマリをそれぞれ生成し、そのサマリのサマリを作ってインプットとする
# map_rerank ... 得られた候補にそれぞれスコアを振って、いちばん高いものをインプットとして回答を得る
# refine  ... 得られた候補のサマリを生成し、次にそのサマリと次の候補の様裏を作ることを繰り返す
chain = load_qa_chain(OpenAI(temperature=0.1,max_tokens=1000), chain_type="stuff")
# chain = load_qa_chain(OpenAI(temperature=0.1,max_tokens=1000,model_name="gpt-4-0613"), chain_type="stuff")
query = "この企業の株式発行数を教えて"
docs = db.similarity_search(query)
# langchainを使って検索
chain.run(input_documents=docs, question=query)

# 5. チャットbotの作成（Option)
ここでは、これまでに読み込んだドキュメントを使って簡易のチャットボットを作成します。
ここを実行するとダイアログが表示されますので、前に読み込ませたドキュメントに関する質問をして見てください。

In [ ]:
from IPython.display import display
import ipywidgets as widgets

# vextordbをretrieverとして使うconversation chainを作成します。これはチャット履歴の管理も可能にします。
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0.1), db.as_retriever())

In [ ]:
chat_history = []

def on_submit(_):
    query = input_box.value
    input_box.value = ""

    if query.lower() == 'exit':
        print("Thank you for using the State of the Union chatbot!")
        return

    result = qa({"question": query, "chat_history": chat_history})
    chat_history.append((query, result['answer']))

    display(widgets.HTML(f'<b>User:</b> {query}'))
    display(widgets.HTML(f'<b><font color="blue">Chatbot:</font></b> {result["answer"]}'))

print("Welcome to the Transformers chatbot! Type 'exit' to stop.")

input_box = widgets.Text(placeholder='Please enter your question:')
input_box.on_submit(on_submit)

display(input_box)